# Analysis script for results in the paper

In [1]:
# import libraries

import scipy
from scipy.stats import sem as sem
import sys
import glob
import pandas as pd

if "/home/anna/Documents/cinnabar" not in sys.path:
    sys.path.insert(1, "/home/anna/Documents/cinnabar")
import cinnabar

print("adding code to the pythonpath...")
code = "/home/anna/Documents/code/python"
if code not in sys.path:
    sys.path.insert(1, code)
import pipeline

print(cinnabar.__file__)

from pipeline import *
from pipeline.utils import validate
from pipeline.analysis import *

adding code to the pythonpath...


INFO:rdkit:Enabling RDKit 2023.03.2 jupyter extensions


/home/anna/Documents/cinnabar/cinnabar/__init__.py


load in the data for the protein system


In [2]:
bench_folder = f"/home/anna/Documents/benchmark"
protein = "tyk2"
main_dir = f"/backup/{protein}"
# main_dir = f"/backup/manual_reruns/{protein}"

ana_obj_dict = {}

# for the different networks
for net in ["lomap", "rbfenn", "combined"]:
    # choose location for the files
    net_file = f"{main_dir}/execution_model/network_{net}.dat"
    ana_file = f"{main_dir}/execution_model/analysis_protocol.dat"  # can also cycle through different analysis protocols
    exp_file = f"{bench_folder}/inputs/experimental/{protein}.yml"

    output_folder = f"{main_dir}/outputs_extracted"

    ana_obj = analysis_network(
        output_folder,
        exp_file=exp_file,
        net_file=net_file,
        analysis_prot=ana_file,
    )

    # Add ligands folder for drawing

    ana_obj.add_ligands_folder(f"{bench_folder}/inputs/{protein}/ligands")

    ana_obj_dict[net] = ana_obj

INFO:root:name not found in protocol. None will be used.
INFO:root:no output folder provided, writing all output to the 'output_folder/analysis'.
INFO:root:name not found in protocol. None will be used.
INFO:root:no output folder provided, writing all output to the 'output_folder/analysis'.
INFO:root:name not found in protocol. None will be used.
INFO:root:no output folder provided, writing all output to the 'output_folder/analysis'.
/home/anna/mambaforge/envs/openbiosim-dev/lib/python3.9/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/anna/mambaforge/envs/openbiosim-dev/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/anna/mambaforge/envs/openbiosim-dev/lib/python3.9/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=

In [3]:
# analyse each object

for ana_obj in ana_obj_dict.values():

    # compute results
    ana_obj.compute_results()

### reproducibility between perturbations

this is for the combined networks

check:
vs experimental
vs each other
outliers for each engine
most different perturbations

In [4]:
ana_obj = ana_obj_dict["combined"]

# compared to each other
mad_df, mad_df_err = ana_obj.calc_mad_engines(pert_val="pert")
print(mad_df)
print(mad_df_err)

# # compared to experimental
mae_df, mae_df_err = ana_obj.calc_mae_engines(pert_val="pert")
print(mae_df)
print(mae_df_err)

# also saved in output folder

            AMBER      SOMD   GROMACS
AMBER         0.0  0.707312  0.925544
SOMD     0.707312       0.0  1.099284
GROMACS  0.925544  1.099284       0.0
            AMBER      SOMD   GROMACS
AMBER         0.0  0.135508  0.134861
SOMD     0.135168       0.0  0.192696
GROMACS  0.135504  0.192492       0.0
                 AMBER      SOMD   GROMACS
experimental  1.071701  1.167511  1.140621
                 AMBER      SOMD  GROMACS
experimental  0.109095  0.142482  0.13287


plot the correlation plots

In [ ]:
ana_obj.plot_scatter_ddG()

for eng in ana_obj.engines:
    ana_obj.plot_scatter_ddG(engines=eng)
    # ana_obj.plot_scatter_ddG(engines=eng, use_cinnabar=True)

checking results, convergence, spread of data between engines

In [ ]:
for ana_obj in ana_obj_dict.values():
    # can compute convergence for all
    ana_obj.compute_convergence(main_dir=main_dir)
    ana_obj.plot_convergence()

In [ ]:
# histograms

for ana_obj in ana_obj_dict.values():
    ana_obj.plot_histogram_repeats()

    ana_obj.plot_histogram_legs()

    ana_obj.plot_histogram_sem()

outliers for each engine

In [ ]:
ana_obj.plot_outliers(no_outliers=5, engines=ana_obj.engines)

In [ ]:
for eng in ana_obj.engines:
    # get outliers above a certain threshold
    perts = ana_obj.get_outliers(threshold=3, name=eng)
    print(f"{eng} : {perts}")

    # draw the perturbations
    ana_obj.draw_perturbations(perts)

perturbations that are the most different between the engines

In [ ]:
all_engines_list_dict = {}

for pert in ana_obj.perturbations:
    all_engines_list_dict[pert] = []

    for eng in ana_obj.engines:
        all_engines_list_dict[pert].append(ana_obj.calc_pert_dict[eng][pert][0])

In [ ]:
def find_top_largest_difference_pert(dictionary_of_lists, num_ranges):
    ranges = {}

    for key, sublist in dictionary_of_lists.items():
        range = max(sublist) - min(sublist)
        ranges[key] = range

    sorted_ranges = sorted(ranges.items(), key=lambda x: x[1], reverse=True)
    top_ranges = sorted_ranges[:num_ranges]

    return top_ranges

In [ ]:
top_ranges = find_top_largest_difference_pert(all_engines_list_dict, 5)
print(top_ranges)
perts = [a[0] for a in top_ranges]
ana_obj.draw_perturbations(perts)

In [ ]:
ana_obj._plotting_object.scatter("pert", values=perts, y_names=ana_obj.engines)

In [ ]:
for eng in ana_obj.engines:
    print(eng)
    for key in ana_obj.calc_pert_dict[eng].items():
        print(key)

### cycle closures

for each network

In [ ]:
cycles

In [ ]:
for net in ana_obj_dict:
    print(net)
    ana_obj = ana_obj_dict[net]

    ana_obj.compute_cycle_closures()

    for eng in ana_obj.cycle_dict:
        print(eng)
        cycles = ana_obj.cycle_dict[eng]
        print(f"{eng} cycle vals is {cycles[1]}")
        print(f"{eng} cycle mean is {cycles[2]}")
        print(f"{eng} cycle deviation is {cycles[3]}")

        max_cycle_ind = max(cycles[1])
        max_cycle = list(cycles[0].keys())[cycles[1].index(max_cycle_ind)]
        print(max_cycle)

### reproducibility between ligs - per ligand results

this is for the individual networks

In [6]:
# plot for each network and calc mad / mae


for net in ana_obj_dict:
    print(net)
    ana_obj = ana_obj_dict[net]

    # plotting with r2, spearmans rank
    # for eng in ana_obj.engines:
        # title = ""
        # title += f"{net}"

        # mue_exp = r2val = stats["val"]["experimental"][eng]["MUE"]
        # r2val = stats["val"]["experimental"][eng]["R2"]
        # rmseval = stats["val"]["experimental"][eng]["RMSE"]
        # spearman = stats["val"]["experimental"][eng]["rho"]

        # # titles
        # title += f"\n MAE : {mue_exp[0]:.2f} +/- {mue_exp[1]:.2f} kcal/mol"
        # title += f"\n R2 : {r2val[0]:.2f} +/- {r2val[1]:.2f} kcal/mol"
        # title += f"\n RMSE : {rmseval[0]:.2f} +/- {rmseval[1]:.2f} kcal/mol"
        # title += f"\n rho : {spearman[0]:.2f} +/- {spearman[1]:.2f} kcal/mol"

        # kwargs = {"title": title}
        # ana_obj.plot_scatter_dG(engine=eng, **kwargs)
        # ana_obj.plot_scatter_dG(engine=eng, use_cinnabar=True)

    # compared to each other
    print("mad")
    mad_df, mad_df_err = ana_obj.calc_mad_engines(pert_val="val")
    print(mad_df)
    print(mad_df_err)

    # # compared to experimental
    print("mae")
    mae_df, mae_df_err = ana_obj.calc_mae_engines(pert_val="val")
    print(mae_df)
    print(mae_df_err)

    # # compared to experimental
    print("ktau")
    mae_df, mae_df_err = ana_obj.calc_kendalls_rank_engines(pert_val="val")
    print(mae_df)
    print(mae_df_err)

    print("spearmans")
    mae_df, mae_df_err = ana_obj.calc_spearmans_rank_engines(pert_val="val")
    print(mae_df)
    print(mae_df_err)

    print("r2")
    mae_df, mae_df_err = ana_obj.calc_r2_engines(pert_val="val")
    print(mae_df)
    print(mae_df_err)
    # also saved in output folder

lomap
mad
            AMBER      SOMD   GROMACS
AMBER         0.0  0.468575   0.53787
SOMD     0.468575       0.0  0.574991
GROMACS   0.53787  0.574991       0.0
            AMBER      SOMD   GROMACS
AMBER         0.0  0.077052  0.087613
SOMD     0.076594       0.0  0.071923
GROMACS  0.088977  0.073158       0.0
mae
                 AMBER      SOMD   GROMACS
experimental  0.707341  0.617231  0.795372
                AMBER      SOMD   GROMACS
experimental  0.09803  0.112863  0.140935
ktau
                 AMBER      SOMD   GROMACS
experimental  0.544118  0.544118  0.485294
                 AMBER      SOMD   GROMACS
experimental  0.113412  0.139907  0.171271
spearmans
                 AMBER      SOMD   GROMACS
experimental  0.793316  0.823745  0.660862
                 AMBER      SOMD   GROMACS
experimental  0.072669  0.075362  0.147771
r2
                AMBER      SOMD   GROMACS
experimental  0.62935  0.678555  0.436738
                 AMBER     SOMD   GROMACS
experimental  0.111845  

### different network analysis methods

all so far with cinnabar.

In [5]:
# compared to fwf

for net in ana_obj_dict:
    print(net)
    ana_obj = ana_obj_dict[net]

    # first need to add the fwf path
    ana_obj._add_fwf_path(
        "/home/anna/Documents/september_2022_workshops/freenrgworkflows/networkanalysis"
    )

    # all_analysis_object._add_fwf_path(
    #     "/home/anna/Documents/freenrgworkflows/networkanalysis"
    # )

    title = ""
    title += f"{net}"

    for eng in ana_obj.engines:
        print(eng)
        # get the network analysis
        fwf_dict = ana_obj._get_ana_fwf(engine=eng)
        # for key in fwf_dict:
        #     print(f"{key} : {fwf_dict[key][0]}, {fwf_dict[key][1]}")

        # get fwf stats
        r_confidence, tau_confidence, mue_confidence = ana_obj._get_stats_fwf(engine=eng)
        print("r: ", r_confidence)
        print("tau: ", tau_confidence)
        print("mae: ", mue_confidence)

    # compared to each other
    mad_df, mad_df_err = ana_obj._get_mad_fwf(ana_obj.engines, ana_obj.engines)
    print(mad_df)
    print(mad_df_err)

lomap
AMBER
Added additional data to 40 edges; added 3 new edges.
Added additional data to 40 edges; added 2 new edges.
r:  [0.75368769 0.73656196 0.77014228]
tau:  [0.51470588 0.5        0.54411765]
mae:  [0.68357111 0.65900032 0.70715194]
SOMD
Added additional data to 43 edges; added 1 new edges.
Added additional data to 43 edges; added 1 new edges.
r:  [0.64302507 0.62311349 0.66156468]
tau:  [0.45588235 0.42647059 0.48529412]
mae:  [0.85167338 0.82856806 0.87519561]
GROMACS
Added additional data to 44 edges; added 0 new edges.
Added additional data to 44 edges; added 0 new edges.
r:  [0.60386437 0.58501265 0.62223302]
tau:  [0.38235294 0.36764706 0.41176471]
mae:  [0.90923182 0.88697873 0.93302571]
Added additional data to 40 edges; added 3 new edges.
Added additional data to 40 edges; added 2 new edges.
Added additional data to 43 edges; added 1 new edges.
Added additional data to 43 edges; added 1 new edges.
Added additional data to 44 edges; added 0 new edges.
Added additional d

In [4]:
# compared to mbarnet

# ana_obj_dict["combined"].analyse_mbarnet(compute_missing=True, use_experimental=True, write_xml=True, run_xml_py=True)

for net in ana_obj_dict:
    print(net)
    ana_obj = ana_obj_dict[net]

    ana_obj.analyse_mbarnet(compute_missing=False, use_experimental=True, write_xml=False, run_xml_py=False)

    statistics = ["MUE", "R2", "rho", "KTAU"]

    for stats in statistics:
        print(stats)
        df,dferr = ana_obj._get_stats_mbarnet(statistic=stats)
        print(df)
        print(dferr)

    # compared to each other
    mad_df, mad_df_err = ana_obj._get_mad_mbarnet(ana_obj.engines, ana_obj.engines)
    print(mad_df)
    print(mad_df_err)


lomap
MUE
                 AMBER       SOMD    GROMACS
experimental  9.821979  10.563101  10.207924
                 AMBER      SOMD   GROMACS
experimental  0.242192  0.770182  0.247066
R2
                 AMBER      SOMD   GROMACS
experimental  0.395059  0.006659  0.400142
                 AMBER      SOMD   GROMACS
experimental  0.175917  0.142144  0.151561
rho
                 AMBER    SOMD   GROMACS
experimental  0.628537 -0.0816  0.632568
                 AMBER      SOMD   GROMACS
experimental  0.154724  0.333896  0.142992
KTAU
                 AMBER      SOMD   GROMACS
experimental  0.383333  0.058824  0.338235
                 AMBER      SOMD   GROMACS
experimental  0.154969  0.235391  0.186774
            AMBER      SOMD   GROMACS
AMBER         0.0  1.757437   0.67275
SOMD     1.757437       0.0  1.620588
GROMACS   0.67275  1.620588       0.0
            AMBER      SOMD   GROMACS
AMBER         0.0  0.661605  0.097141
SOMD     0.647717       0.0  0.578845
GROMACS   0.09724  0.569

plotting just fwf data per ligand

In [ ]:
dict_y = fwf_dict
dict_exp = exp_dicts[0]

df1 = plotting_engines.match_dicts_to_df(dict_exp, dict_y, "experimental", "fwf")
df1

df1.plot.bar(
    y=["freenrg_fwf", "freenrg_experimental"],
    yerr=df1[["err_fwf", "err_experimental"]].T.values,
    title=f"fwf, experimental, {eng}",
    xlabel="ligands",
    ylabel="dG (kcal/mol)",
)

df1.dropna()
df1.plot.scatter(
    x="freenrg_experimental",
    y="freenrg_fwf",
    xerr="err_experimental",
    yerr="err_fwf",
    title=f"fwf, {eng}",
    xlabel="experimental dG (kcal/mol)",
    ylabel="fwf dG (kcal/mol)",
)

# calculating using the cinnabar stats
f_mae = all_analysis_object._stats_object._compute_stats(
    x=df1["freenrg_experimental"],
    y=df1["freenrg_fwf"],
    xerr=df1["err_experimental"],
    yerr=df1["err_fwf"],
    statistic="MUE",
)
print(f_mae)

### consensus

consensus scoring of the engines

In [ ]:
# get average of averages

for net in ana_obj_dict:
    ana_obj = ana_obj_dict[net]

    consensus_pert_dict = {}

    ana_obj.compute_consensus()

    print(net)

In [ ]:
# consensus scoring, is it more robust

for net in ana_obj_dict:
    print(net)

    ana_obj = ana_obj_dict[net]

    for pv in ["pert", "val"]:
        print(pv)
        mae_df, mae_df_err = ana_obj.calc_mae_engines(pv, engines="consensus")
        print(mae_df)
        print(mae_df_err)

        stat_rank = ana_obj._stats_object.compute_rho(pv, y="consensus")
        print(stat_rank)

### comparing different networks

for the lomap and the rbfenn 

In [ ]:
error_val_dict.values()

In [ ]:
# check if the error for the different networks is lower

error_dict = {}
diff_to_exp_dict = {}

for net in ana_obj_dict:
    print(net)

    ana_obj = ana_obj_dict[net]

    error_dict[net] = {}
    diff_to_exp_dict[net] = {}

    for eng in ana_obj.engines:
        error_val_dict = {}
        diff_to_exp_val_dict = {}

        for key in ana_obj.calc_pert_dict[eng]:
            error_val_dict[key] = ana_obj.calc_pert_dict[eng][key][1]
            diff_to_exp_val_dict[key] = abs(
                ana_obj.calc_pert_dict[eng][key][0] - ana_obj.exper_pert_dict[key][0]
            )

        # test for normal distribution in the errors
        # if less than 0.05, not normal
        print(
            "error",
            scipy.stats.shapiro(
                list(filter(lambda item: item is not None, error_val_dict.values()))
            ),
        )
        print(
            "diff to exp",
            scipy.stats.shapiro(
                list(
                    filter(lambda item: item is not None, diff_to_exp_val_dict.values())
                )
            ),
        )

        avg_error = np.mean(
            list(filter(lambda item: item is not None, error_val_dict.values()))
        )
        avg_diff_exp = np.mean(
            list(filter(lambda item: item is not None, diff_to_exp_val_dict.values()))
        )

        print(f"{net}, {eng}, avg error is : {avg_error}")
        print(f"{net}, {eng}, avg diff to exp is : {avg_diff_exp}")

        error_dict[net][eng] = error_val_dict
        diff_to_exp_dict[net][eng] = diff_to_exp_val_dict

In [ ]:
# mann whitney u, as not normally distributed

for eng in ana_obj.engines:
    group1 = list(
        filter(lambda item: item is not None, error_dict["lomap"][eng].values())
    )
    group2 = list(
        filter(lambda item: item is not None, error_dict["rbfenn"][eng].values())
    )

    ustats, pvalue = scipy.stats.mannwhitneyu(group1, group2)

    print(f"{eng}: {ustats, pvalue}")

# if below 0.05 (if confidence interval) there is significant difference (reject null hypothesis)

In [ ]:
# check difference to experimental sig diff between the two

for eng in ana_obj.engines:
    group1 = list(
        filter(lambda item: item is not None, diff_to_exp_dict["lomap"][eng].values())
    )
    group2 = list(
        filter(lambda item: item is not None, diff_to_exp_dict["rbfenn"][eng].values())
    )

    ustats, pvalue = scipy.stats.mannwhitneyu(group1, group2)

    print(f"{eng}: {ustats, pvalue}")

### directionality

data from featurising the perturbations

In [ ]:
ana_obj = ana_obj_dict["combined"]

grow_shrink_dict = {}

for eng in ana_obj.engines:
    grow_shrink_dict[eng] = {}

    error_dict = {
        key: ana_obj.calc_pert_dict[eng][key][1] for key in ana_obj.calc_pert_dict[eng]
    }
    df = pd.read_csv(f"{main_dir}/execution_model/grow_shrink_featurise.dat")
    df[f"error_{eng}"] = df["pert"].map(error_dict)
    df = df.dropna()

    group1 = df.loc[df["grow/shrink"] == "grow"][f"error_{eng}"]
    group2 = df.loc[df["grow/shrink"] == "shrink"][f"error_{eng}"]
    ustats, pvalue = scipy.stats.mannwhitneyu(group1, group2)
    print(f"mann u for error {eng}: {ustats, pvalue}")
    print(
        f"mean for error {eng} grow: {np.mean(group1)}, and for shrink: {np.mean(group2)}"
    )

    grow_shrink_dict[eng]["grow_err"] = group1
    grow_shrink_dict[eng]["shrink_err"] = group2

    # for diff to experimental
    diff_dict = {
        key: diff_to_exp_dict["combined"][eng][key]
        for key in ana_obj.calc_pert_dict[eng]
    }
    df[f"diff_{eng}"] = df["pert"].map(diff_dict)
    df = df.dropna()

    group1 = df.loc[df["grow/shrink"] == "grow"][f"diff_{eng}"]
    group2 = df.loc[df["grow/shrink"] == "shrink"][f"diff_{eng}"]
    ustats, pvalue = scipy.stats.mannwhitneyu(group1, group2)
    print(f"mann u for diff to exp {eng}: {ustats, pvalue}")
    print(
        f"mean for diff to exp {eng} grow: {np.mean(group1)}, and for shrink: {np.mean(group2)}"
    )

    grow_shrink_dict[eng]["grow_diff"] = group1
    grow_shrink_dict[eng]["shrink_diff"] = group2

# if below 0.05 (if confidence interval) there is significant difference (reject null hypothesis)

In [ ]:
# different between engines significant?

res_dict = {}

for size in ["grow_err", "shrink_err", "grow_diff", "shrink_diff"]:
    res_dict[size] = {}

    for eng in ana_obj_dict["combined"].engines:
        res_dict[size][eng] = {}

    for combo in it.product(grow_shrink_dict.keys(), grow_shrink_dict.keys()):
        eng1 = combo[0]
        eng2 = combo[1]

        if eng1 == eng2:
            continue

        group1 = grow_shrink_dict[eng1][size]
        group2 = grow_shrink_dict[eng2][size]

        ustats, pvalue = scipy.stats.mannwhitneyu(group1, group2)
        print(f"{eng1, eng2}, {size}: {ustats, pvalue}")
        print(f"mean for {eng1}: {np.mean(group1)}, and for {eng2}: {np.mean(group2)}")

        res_dict[size][eng1][eng2] = pvalue

In [ ]:
df = pd.DataFrame.from_dict(res_dict["grow_err"])
df

size of perturbation and variability

In [ ]:
ana_obj = ana_obj_dict["combined"]

file = f"{bench_folder}/extracted/{protein}/perturbing_overlap.dat"

for eng in ana_obj.engines:
    df = pd.read_csv(file)

    df = df[df["engine"] == eng]

    error_dict = {
        key: ana_obj.calc_pert_dict[eng][key][1] for key in ana_obj.calc_pert_dict[eng]
    }
    df[f"error"] = df["perturbation"].map(error_dict)
    # df = df.dropna()
    df = df[df["error"].notna()]
    df = df[df["perturbing_atoms"].notna()]

    stats = pipeline.analysis.stats_engines.compute_stats(
        [x for x in df["perturbing_atoms"]], [x for x in df["error"]], statistic="R2"
    )
    df.plot.scatter(
        "perturbing_atoms",
        "error",
        c="diff_to_exp",
        colormap="viridis",
        title=f"{eng}\n{stats}",
    )

### different analysis methods

the different analysis methods (MBAR/TI, % of run used, stats ineff, autoeq)

In [ ]:
# add other analyses as other results

ana_obj = ana_obj_dict["combined"]

for eng in ana_obj.engines:
    other_results = glob.glob(
        f"{results_folder}/freenrg_*_{eng}_TI_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
    )
    bound_results = glob.glob(
        f"{results_folder}/bound_*_{eng}_TI_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
    )
    free_results = glob.glob(
        f"{results_folder}/free_*_{eng}_TI_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
    )
    ana_obj.compute_other_results(
        other_results,
        name=f"{eng}_TI",
        bound_files=bound_results,
        free_files=free_results,
    )

# for eng in ana_obj.engines:

#     other_results = glob.glob(
#         f"{results_folder}/freenrg_*_{eng}_MBAR_alchemlyb_None_eqtrue_statstrue_truncate0end.csv"
#     )
#     bound_results = glob.glob(
#         f"{results_folder}/bound_*_{eng}_MBAR_alchemlyb_None_eqtrue_statstrue_truncate0end.csv"
#     )
#     free_results = glob.glob(
#         f"{results_folder}/free_*_{eng}_MBAR_alchemlyb_None_eqtrue_statstrue_truncate0end.csv"
#     )
#     ana_obj.compute_other_results(
#         other_results, name=f"{eng}_MBAR_stats_eq", bound_files=bound_results, free_files=free_results
#     )

# for eng in ana_obj.engines:

#     other_results = glob.glob(
#         f"{results_folder}/freenrg_*_{eng}_MBAR_alchemlyb_None_eqfalse_statstrue_truncate0end.csv"
#     )
#     bound_results = glob.glob(
#         f"{results_folder}/bound_*_{eng}_MBAR_alchemlyb_None_eqfalse_statstrue_truncate0end.csv"
#     )
#     free_results = glob.glob(
#         f"{results_folder}/free_*_{eng}_MBAR_alchemlyb_None_eqfalse_statstrue_truncate0end.csv"
#     )
#     ana_obj.compute_other_results(
#         other_results, name=f"{eng}_MBAR_stats", bound_files=bound_results, free_files=free_results
# )

In [ ]:
for eng in ana_obj.engines:
    stat_r2 = ana_obj._stats_object.compute_r2("pert", f"{eng}_TI", f"{eng}")
    stat_r2_string = f"{stat_r2[0]:.2f} +/- {stat_r2[1]:.2f}"
    # ana_obj.plot_eng_vs_eng(engine_a=f"{eng}_TI", engine_b=f"{eng}", **{"title":f"{eng} vs {eng}_TI\n{stat_r2_string}"})

    ana_obj._plotting_object.scatter(
        pert_val="pert",
        y_names=f"{eng}_TI",
        x_name=f"{eng}",
        outliers=True,
        no_outliers=3,
        **{"title": f"{eng} vs {eng}_TI\n{stat_r2_string}"},
    )